In [775]:
# Suzan Iloglu, May 21,2020
# Import packages
import csv
import gurobipy as gp
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
import requests
import io
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200
from IPython.display import Image


# MAPPING THE NEW POLITICS OF CARE: COMMUNITY HEALTH WORKERS
The project presents multiple options for how individual workers in such a Community Health Corps might be distributed within each state. It shows that what you choose to prioritize greatly impacts where care would be sent. We can define communities in greatest need in many ways: we can think about our current crisis and send people to where the COVID19 pandemic rages most fiercely; we can think of long term measures of social and economic inequality embedded in metrics like the Centers for Disease Control and Prevention’s Social Vulnerability Index; we can focus on the places with too many people dying too young and use the County Health Rankings Years-of-Potential-Life-Lost measure; we can think of joblessness and how the pandemic has thrown many into unemployment and target our resources in this way. 

The followings are our options to choose to define vulnerability:


- SOCIAL VULNERABILITY INDEX
- MEDICAID 
- UNEMPLOYMENT
- YEARS OF POTENTIAL LIFE LOST
- TOTAL COVID CASES
- COVID CASES BY POPULATION
- COVID DEATHS BY POPULATION

We will start with Social Vulnerability Index (SVI) from CDC website.

### I. Importing SVI data which includes the variables for calculating county SVI for each state
The CDC uses both a USA-wide and a state by state SVI scores. For our project given that funding is likely going to be managed at a state level, using a state by state SVI scores makes the most sense and will be most sensitive to regional socioeconomic differences. Even though the CDC SVI scores are calculated using percentile rankings, the data sets include raw data estimates for each variables. The following table shows the variablaes used in the method of calculating SVI scores. 




      American Community Survey (ACS), 2014-2018 (5-year) data for the following estimates:
<img src="Data/img/SVI_comp.png" width="500">


Note: Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a> 
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Later in the notebook, we will provide the formula to create the SVI value we use in our project. First, we import the data for the US mainland and Puerto Rico.

In [776]:
## import svi data downloaded from CDC website as cited above

## 48 state SVI scores by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")

## Puerto Rico SVI scores by county
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")

## Merge 48 states and Puerto Rico SVI 
svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)


In [777]:
## Replacing -999 values with 0 for calculations
svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)

In [778]:
## Create the list for State
State = svi_county.STATE.unique().tolist()

In [779]:
# Create a seperate dictionary for the variables to calculate SVI

# Persons below poverty estimate, 2014-2018 ACS
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Civilian (age 16+) unemployed estimate, 2014-2018 ACS
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.E_UNEMP))

# Per capita income estimate, 2014-2018 ACS
E_PCI = dict(zip(svi_county.FIPS, svi_county.E_PCI))

# Persons (age 25+) with no high school diploma estimate, 2014-2018 ACS
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.E_NOHSDP))

# Persons aged 65 and older estimate
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.E_AGE65))

# Persons aged 17 and younger estimate
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.E_AGE17))

# Population with a disability estimate
E_DISABL = dict(zip(svi_county.FIPS, svi_county.E_DISABL))

# Single parent households with children under 18 estimate
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.E_SNGPNT))

# Minority (all persons except white, nonHispanic) estimate, 2014-2018 ACS
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.E_MINRTY))

# Persons (age 5+) who speak English "less than well" estimate, 2014-2018 ACS
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.E_LIMENG))

# Housing in structures with 10 or more units estimate, 2014-2018 ACS
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.E_MUNIT))

# Mobile homes estimate MOE, 2014-2018 ACS
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.E_MOBILE))

# At household level (occupied housing units), more people than rooms estimate, 2014-2018 ACS
E_CROWD = dict(zip(svi_county.FIPS, svi_county.E_CROWD))

# Households with no vehicle available estimate, 2014-2018 ACS
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.E_NOVEH))

# Persons in institutionalized group quarters estimate, 2014-2018 ACS
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.E_GROUPQ))

# Percentage of persons below poverty estimate
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Medicaid 
Medicaid is a means-tested health insurance program for low-income children, pregnant women, adults, seniors, and people with disabilities. Medicaid is jointly funded by federal and state governments and managed by states within federal standards and a wide range of state options. <a href="https://data.medicaid.gov/Enrollment/State-Medicaid-and-CHIP-Applications-Eligibility-D/n5ce-jxme"> Data Source for Medicaid Enrollment </a> 

In [780]:

import sodapy
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.medicaid.gov", None)


# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("83yt-67it", limit=4000)


# Read the medicaid demand data
#df_mm = pd.read_csv("Data/2020_06_Preliminary_applications__eligibility_determinations__and_enrollment_data.csv")

# Convert to pandas DataFrame
df_mm = pd.DataFrame.from_records(results)
df_mm.head(5)
df_mm.columns


Index(['applications_for_financial_assistance_submitted_to_the_state_based_marketplace',
       'applications_for_financial_assistance_submitted_to_the_state_based_marketplace_footnotes',
       'final_report', 'geocoded_column',
       'individuals_determined_eligible_for_chip_at_application',
       'individuals_determined_eligible_for_chip_at_application_footnotes',
       'individuals_determined_eligible_for_medicaid_at_application',
       'individuals_determined_eligible_for_medicaid_at_application_footnotes',
       'latitude', 'longitude', 'medicaid_and_chip_child_enrollment',
       'medicaid_and_chip_child_enrollment_footnotes',
       'new_applications_submitted_to_medicaid_and_chip_agencies',
       'new_applications_submitted_to_medicaid_and_chip_agencies_footnotes',
       'preliminary_updated', 'report_date', 'state_abbreviation',
       'state_expanded_medicaid', 'state_name',
       'total_applications_for_financial_assistance_submitted_at_state_level',
       'total_a

In [781]:
df_mm['State Name'] = df_mm['state_name'].str.upper() 

In [782]:
Medicaid_state = dict(zip(df_mm['State Name'], df_mm['total_medicaid_and_chip_enrollment']))
Medicaid_state['PUERTO RICO'] = 1622194
print (Medicaid_state)

{'ALABAMA': '957116', 'ALASKA': '231145', 'ARIZONA': '1839932', 'ARKANSAS': '830467', 'CALIFORNIA': '11847711', 'COLORADO': '1337805', 'CONNECTICUT': '874974', 'DELAWARE': '239009', 'DISTRICT OF COLUMBIA': '248591', 'FLORIDA': '3892552', 'GEORGIA': '1928703', 'HAWAII': '351337', 'IDAHO': '340742', 'ILLINOIS': '2987496', 'INDIANA': '1602976', 'IOWA': '699741', 'KANSAS': '401103', 'KENTUCKY': '1416013', 'LOUISIANA': '1585024', 'MAINE': '232455', 'MARYLAND': '1372695', 'MASSACHUSETTS': '1616404', 'MICHIGAN': '2439425', 'MINNESOTA': '1085778', 'MISSISSIPPI': '632427', 'MISSOURI': '923641', 'MONTANA': '247333', 'NEBRASKA': '254159', 'NEVADA': '685073', 'NEW HAMPSHIRE': '193436', 'NEW JERSEY': '1759653', 'NEW MEXICO': '772102', 'NEW YORK': '6263164', 'NORTH CAROLINA': '1851558', 'NORTH DAKOTA': '96757', 'OHIO': '2788134', 'OKLAHOMA': '797220', 'OREGON': '1053931', 'PENNSYLVANIA': '3069309', 'RHODE ISLAND': '305208', 'SOUTH CAROLINA': '1048276', 'SOUTH DAKOTA': '114059', 'TENNESSEE': '1489536

In [783]:
df_mmm = pd.read_csv("Data/ACSST5Y2018.S2704_data_with_overlays_2020-08-01T140649.csv", header=[1])
df_mmm.head(1)
#df_mmm.dtypes

,id,Geographic Area Name,Estimate!!Total!!Civilian noninstitutionalized population,Margin of Error!!Total MOE!!Civilian noninstitutionalized population,Estimate!!Public Coverage!!Civilian noninstitutionalized population,Margin of Error!!Public Coverage MOE!!Civilian noninstitutionalized population,Estimate!!Percent Public Coverage!!Civilian noninstitutionalized population,Margin of Error!!Percent Public Coverage MOE!!Civilian noninstitutionalized population,Estimate!!Total!!Medicare coverage alone or in combination,Margin of Error!!Total MOE!!Medicare coverage alone or in combination,Estimate!!Public Coverage!!Medicare coverage alone or in combination,Margin of Error!!Public Coverage MOE!!Medicare coverage alone or in combination,Estimate!!Percent Public Coverage!!Medicare coverage alone or in combination,Margin of Error!!Percent Public Coverage MOE!!Medicare coverage alone or in combination,Estimate!!Total!!Medicare coverage alone or in combination!!Under 19,Margin of Error!!Total MOE!!Medicare coverage alone or in combination!!Under 19,Estimate!!Public Coverage!!Medicare coverage alone or in combination!!Under 19,Margin of Error!!Public Coverage MOE!!Medicare coverage alone or in combination!!Under 19,Estimate!!Percent Public Coverage!!Medicare coverage alone or in combination!!Under 19,Margin of Error!!Percent Public Coverage MOE!!Medicare coverage alone or in combination!!Under 19,Estimate!!Total!!Medicare coverage alone or in combination!!19 to 64 years,Margin of Error!!Total MOE!!Medicare coverage alone or in combination!!19 to 64 years,Estimate!!Public Coverage!!Medicare coverage alone or in combination!!19 to 64 years,Margin of Error!!Public Coverage MOE!!Medicare coverage alone or in combination!!19 to 64 years,Estimate!!Percent Public Coverage!!Medicare coverage alone or in combination!!19 to 64 years,Margin of Error!!Percent Public Coverage MOE!!Medicare coverage alone or in combination!!19 to 64 years,Estimate!!Total!!Medicare coverage alone or in combination!!65 years and over,Margin of Error!!Total MOE!!Medicare coverage alone or in combination!!65 years and over,Estimate!!Public Coverage!!Medicare coverage alone or in combination!!65 years and over,Margin of Error!!Public Coverage MOE!!Medicare coverage alone or in combination!!65 years and over,Estimate!!Percent Public Coverage!!Medicare coverage alone or in combination!!65 years and over,Margin of Error!!Percent Public Coverage MOE!!Medicare coverage alone or in combination!!65 years and over,Estimate!!Total!!Medicaid/means-tested public coverage alone or in combination,Margin of Error!!Total MOE!!Medicaid/means-tested public coverage alone or in combination,Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination,Margin of Error!!Public Coverage MOE!!Medicaid/means-tested public coverage alone or in combination,Estimate!!Percent Public Coverage!!Medicaid/means-tested public coverage alone or in combination,Margin of Error!!Percent Public Coverage MOE!!Medicaid/means-tested public coverage alone or in combination,Estimate!!Total!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Margin of Error!!Total MOE!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Margin of Error!!Public Coverage MOE!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Estimate!!Percent Public Coverage!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Margin of Error!!Percent Public Coverage MOE!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Estimate!!Total!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Margin of Error!!Total MOE!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Marg

In [784]:
df_mmm['FIPS'] = df_mmm.id.astype(str).str[9:]
df_mmm.head(5)
df_mmm.FIPS.astype(int)
df_mmm['FIPS'] = pd.to_numeric(df_mmm['FIPS'])

In [785]:
ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination"]))

In [786]:

#df_m = pd.read_csv("Data/Medicaid_Demand.csv")

# Unemployment 
The unemployment rate is calculated by the U.S. Bureau of Labor Statistics as the percentage of the civilian labor force who are without jobs and have actively sought work within the past four weeks. <a href="https://www.bls.gov/lau/laufaq.htm#Q01"> Data Source for Unemployment  </a> 

In [787]:
from io import StringIO
import datetime 
from datetime import date
from dateutil.relativedelta import relativedelta

back = date.today() + relativedelta(months= -3)
three_months_ago = back.strftime('%b-%y') 


url = 'https://www.bls.gov/web/metro/laucntycur14.txt'
s = requests.get(url).text



df_unemp = pd.read_csv(StringIO(s), sep='|',  skiprows=7, skipfooter=6, engine='python', names = ['LAUS Area Code', 'FIPS State', 'FIPS County', 'Area Title', 'Period', 'Civilian Labor Force','Employed','Unemployed_Level','Unemployed_Rate'])

df_unemp['Period'] = df_unemp['Period'].astype(str)

df_unemp['FIPS'] = df_unemp['LAUS Area Code'].str[3:8]

df_unemp = df_unemp[df_unemp['Period'].str.contains(str(three_months_ago))]

df_unemp.head(5)
#df_unemp.dtypes

,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
38627,CN0100100000000,1,1,"Autauga County, AL",Aug-20,"25,446","24,272","1,174",4.6,01001
38628,CN0100300000000,1,3,"Baldwin County, AL",Aug-20,"98,267","93,297","4,970",5.1,01003
38629,CN0100500000000,1,5,"Barbour County, AL",Aug-20,"9,476","8,806",670,7.1,01005
38630,CN0100700000000,1,7,"Bibb County, AL",Aug-20,"8,676","8,141",535,6.2,01007
38631,CN0100900000000,1,9,"Blount County, AL",Aug-20,"24,792","23,889",903,3.6,01009


In [788]:
#df_unemp[[ 'FIPS', 'Area Title', 'Period','Unemployed_Level','Unemployed_Rate']].to_csv('Data/County_employment.csv', index=False)


In [789]:


df_unemp.replace({'-', 0})

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.replace(',', '')

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].astype(str)

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.strip()

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].str.strip()

df_unemp['FIPS'] = pd.to_numeric(df_unemp['FIPS'])

df_unemp['Unemployed_Level'] = pd.to_numeric(df_unemp['Unemployed_Level'])

df_unemp['Unemployed_Rate'] = pd.to_numeric(df_unemp['Unemployed_Rate'])




# Fill NA with 0
#df_unemp = df_unemp.fillna(0)
df_unemp.tail(5)
#df_unemp.dtypes


,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
41841,CN7214500000000,72,145,"Vega Baja Municipio, PR",Aug-20,"12,806","11,317",1489,11.6,72145
41842,CN7214700000000,72,147,"Vieques Municipio, PR",Aug-20,"2,487","2,230",257,10.3,72147
41843,CN7214900000000,72,149,"Villalba Municipio, PR",Aug-20,"6,722","6,096",626,9.3,72149
41844,CN7215100000000,72,151,"Yabucoa Municipio, PR",Aug-20,"8,064","7,286",778,9.6,72151
41845,CN7215300000000,72,153,"Yauco Municipio, PR",Aug-20,"9,159","8,203",956,10.4,72153


In [790]:
 
## Read the Unemployment data using cvs data 
#df_unemp = pd.read_csv("Data/Unemployment.csv")

# Fill NA with 0
#df_unemp = df_unemp.fillna(0)

#df_unemp.head(5)

# Years of Potential Life Lost (YPLL)

Years of Potential Life Lost (YPLL) measures the rate of premature deaths by region. YPLL is calculated as the sum of the estimated number of years that individuals would have lived if they had not died before the age of 75 per 100,000 people. <a href="https://www.countyhealthrankings.org/sites/default/files/media/document/2020%20County%20Health%20Rankings%20Data%20-%20v2.xlsx"> Data Source for YPLL.  </a> More information about YPLL can be dounf in this <a href="https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources/county-health-rankings-model/health-outcomes/length-of-life/premature-death-ypll"> link. </a> 


In [791]:
# Read the YPLL data
df_y = pd.read_csv("Data/YPLL.csv")

# Fill NA with the mean of the data
df_y = df_y.fillna(df_y.mean())


# Population

In [792]:
# Read the Population data
df_pop = pd.read_csv("Data/County_pop_2019.csv")

# Fill NA with 0
df_pop = df_pop.fillna(0)


In [793]:
# Create a dictionary for the county and population
population_county = df_pop.set_index('FIPS')['pop'].to_dict()

# Create a dictionary for the county and YPLL
YPLL = dict(zip(df_y.FIPS, df_y.YPLL))

# Create a dictionary for the county and Unemployment
Unemployment = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Level))


# Create a dictionary for the county and Community Health Workers (CHW) demand
# Note that we assume a CHW can serve 55 Medicaid patient so the demand for CHW will be

#Medicaid_demand = dict(zip(df_m.FIPS, df_m.Med_Demand))


In [794]:
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

# COVID-19 Cases & COVID-19 Cases per Capita

What are COVID-19 Cases and COVID-19 Cases per Capita?

COVID-19 cases is an absolute metric of the total number of COVID-19 cases in a county over the last fourteen days.  COVID-19 cases per 100,000 is a relative metric calculated by dividing the number of COVID-19 cases by the estimated county population and multiplying by 100,000.  Cases include both confirmed cases, based on viral testing, and probable cases, based on specific criteria for symptoms and epidemiological exposure. We use NY Times Covid data. 


In [795]:
#### Data with the most recent date in NY Times dataset:

today = time.strftime('%Y-%m-%d')
covid_data_update_date = today#'2020-07-21'#today #or enter a specific date such as '2020-07-06'


In [796]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [797]:

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [798]:
covid.tail(50)

,date,county,state,fips,cases,deaths
725731,2020-11-12,Ozaukee,Wisconsin,55089.0,3912,32
725732,2020-11-12,Pepin,Wisconsin,55091.0,304,1
725733,2020-11-12,Pierce,Wisconsin,55093.0,1575,9
725734,2020-11-12,Polk,Wisconsin,55095.0,1309,5
725735,2020-11-12,Portage,Wisconsin,55097.0,3854,32
725736,2020-11-12,Price,Wisconsin,55099.0,557,3
725737,2020-11-12,Racine,Wisconsin,55101.0,11735,131
725738,2020-11-12,Richland,Wisconsin,55103.0,679,10
725739,2020-11-12,Rock,Wisconsin,55105.0,7396,60
725740,2020-11-12,Rusk,Wisconsin,55107.0,521,5


Note: Since NY data is seperately available, we first read the NY data for all 5 different borough then combine with the rest of US data.

In [799]:
# URL for NY
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
#"https://raw.githubusercontent.com/nychealth/coronavirus-data/master/data-by-day.csv"

ny = requests.get(url).content
covid_ny = pd.read_csv(io.StringIO(ny.decode('utf-8')))


covid_ny.tail(5)

,date_of_interest,CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,DEATH_COUNT_PROBABLE,CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,INCOMPLETE
251,11/06/2020,946,51,9,1,808,53,9,168,7,3,133,10,2,211,17,2,249,18,5,171,14,0,135,10,0,279,12,3,204,12,2,117,1,1,87,4,0,6000
252,11/07/2020,725,53,8,3,856,55,10,119,15,1,142,11,2,193,8,3,261,18,5,123,7,0,145,10,0,218,20,3,218,13,2,72,3,1,91,4,1,6000
253,11/08/2020,666,47,5,3,884,55,9,85,9,2,146,11,2,190,8,0,257,16,4,108,6,0,151,9,0,198,14,2,233,14,2,85,10,1,97,5,1,6000
254,11/09/2020,1073,62,3,2,905,57,8,158,15,0,147,12,2,294,19,1,251,16,3,180,6,0,157,9,0,314,18,2,248,15,2,127,4,0,103,5,1,6000
255,11/10/2020,722,25,2,5,888,51,6,115,4,1,142,10,2,219,8,0,241,15,2,130,2,0,155,8,0,218,5,1,254,13,2,40,6,0,96,5,1,6000


In [800]:
Kings = covid_ny[['date_of_interest', 'BK_CASE_COUNT', 'BK_DEATH_COUNT']]
Kings.rename(columns = {'BK_CASE_COUNT': 'd_cases', 'date_of_interest': 'date', 'BK_DEATH_COUNT': 'deaths'} , inplace=True)
Kings['county'] = 'Kings'
Kings['state'] = 'New York'
Kings['fips'] = 36047.0
#Kings.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [801]:
Bronx = covid_ny[['date_of_interest', 'BX_CASE_COUNT', 'BX_DEATH_COUNT']]
Bronx.rename(columns = {'BX_CASE_COUNT': 'd_cases', 'date_of_interest': 'date', 'BX_DEATH_COUNT': 'deaths'} , inplace=True)
Bronx['state'] = 'New York'
Bronx['county'] = 'Bronx'
Bronx['fips'] = 36005.0
#Bronx.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [802]:
Manhattan = covid_ny[['date_of_interest', 'MN_CASE_COUNT', 'MN_DEATH_COUNT']]
Manhattan.rename(columns = {'MN_CASE_COUNT': 'd_cases', 'date_of_interest': 'date', 'MN_DEATH_COUNT': 'deaths'} , inplace=True)
Manhattan['state'] = 'New York'
Manhattan['county'] = 'Manhattan'
Manhattan['fips'] = 36061.0
#Manhattan.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [803]:
Queens = covid_ny[['date_of_interest', 'QN_CASE_COUNT', 'QN_DEATH_COUNT']]
Queens.rename(columns = {'QN_CASE_COUNT': 'd_cases', 'date_of_interest': 'date', 'QN_DEATH_COUNT': 'deaths'} , inplace=True)
Queens['state'] = 'New York'
Queens['county'] = 'Queens'
Queens['fips'] = 36081.0
#Queens.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [804]:
Richmond = covid_ny[['date_of_interest', 'SI_CASE_COUNT', 'SI_DEATH_COUNT']]
Richmond.rename(columns = {'SI_CASE_COUNT': 'd_cases', 'date_of_interest': 'date', 'SI_DEATH_COUNT': 'deaths'} , inplace=True)
Richmond['state'] = 'New York'
Richmond['county'] = 'Richmond'
Richmond['fips'] = 36085.0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [805]:
covid.head(5)


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [806]:
Bronx.tail(5)

,date,d_cases,deaths,state,county,fips
251,11/06/2020,168,3,New York,Bronx,36005.0
252,11/07/2020,119,1,New York,Bronx,36005.0
253,11/08/2020,85,2,New York,Bronx,36005.0
254,11/09/2020,158,0,New York,Bronx,36005.0
255,11/10/2020,115,1,New York,Bronx,36005.0


In [807]:
#Kings['deaths'] = Kings.groupby(by=['fips'])['deaths_d'].sum()
#Bronx['deaths'] = Bronx.groupby(by=['fips'])['deaths_d'].sum()
#Manhattan['deaths'] = Manhattan.groupby(by=['fips'])['deaths_d'].sum()
#Queens['deaths'] = Queens.groupby(by=['fips'])['deaths_d'].sum()
#Richmond['deaths'] = Richmond.groupby(by=['fips'])['deaths_d'].sum()

In [808]:
Kings.head(5)

,date,d_cases,deaths,county,state,fips
0,02/29/2020,0,0,Kings,New York,36047.0
1,03/01/2020,0,0,Kings,New York,36047.0
2,03/02/2020,0,0,Kings,New York,36047.0
3,03/03/2020,0,0,Kings,New York,36047.0
4,03/04/2020,1,0,Kings,New York,36047.0


In [809]:
#Kings = Kings.drop(['deaths_d'], axis=1)
#Bronx = Bronx.drop(['deaths_d'], axis=1)
#Manhattan = Manhattan.drop(['deaths_d'], axis=1)
#Queens = Queens.drop(['deaths_d'], axis=1)
#Richmond = Richmond.drop(['deaths_d'], axis=1)

In [810]:
Kings.tail(5)

,date,d_cases,deaths,county,state,fips
251,11/06/2020,211,2,Kings,New York,36047.0
252,11/07/2020,193,3,Kings,New York,36047.0
253,11/08/2020,190,0,Kings,New York,36047.0
254,11/09/2020,294,1,Kings,New York,36047.0
255,11/10/2020,219,0,Kings,New York,36047.0


In [811]:
covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)
Kings['dt'] = pd.to_datetime(Kings['date'], infer_datetime_format=True)
Bronx['dt'] = pd.to_datetime(Bronx['date'], infer_datetime_format=True)
Manhattan['dt'] = pd.to_datetime(Manhattan['date'], infer_datetime_format=True)
Queens['dt'] = pd.to_datetime(Queens['date'], infer_datetime_format=True)
Richmond['dt'] = pd.to_datetime(Richmond['date'], infer_datetime_format=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [813]:

Kings_c = Kings[(Kings['dt']>date_N1_days_ago) & (Kings['dt']<= data_date_dt)].copy()
Queens_c = Queens[(Queens['dt']>date_N1_days_ago) & (Queens['dt']<= data_date_dt)].copy()
Bronx_c = Bronx[(Bronx['dt']>date_N1_days_ago) & (Bronx['dt']<= data_date_dt)].copy()
Manhattan_c = Manhattan[(Manhattan['dt']>date_N1_days_ago) & (Manhattan['dt']<= data_date_dt)].copy()
Richmond_c = Richmond[(Richmond['dt']>date_N1_days_ago) & (Richmond['dt']<= data_date_dt)].copy()

In [821]:
Kings['cases'] = Kings.loc[(Kings['dt']<= data_date_dt)].groupby(['dt'])['d_cases'].sum()
Kings['cases'] = Kings.apply(lambda x: x[(Kings['dt']<= data_date_dt)]['d_cases'].sum())

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [822]:
Kings.head(50)

,date,d_cases,deaths,county,state,fips,dt,cases
0,02/29/2020,0,0,Kings,New York,36047.0,2020-02-29,NaN
1,03/01/2020,0,0,Kings,New York,36047.0,2020-03-01,NaN
2,03/02/2020,0,0,Kings,New York,36047.0,2020-03-02,NaN
3,03/03/2020,0,0,Kings,New York,36047.0,2020-03-03,NaN
4,03/04/2020,1,0,Kings,New York,36047.0,2020-03-04,NaN
5,03/05/2020,3,0,Kings,New York,36047.0,2020-03-05,NaN
6,03/06/2020,1,0,Kings,New York,36047.0,2020-03-06,NaN
7,03/07/2020,2,0,Kings,New York,36047.0,2020-03-07,NaN
8,03/08/2020,5,0,Kings,New York,36047.0,2020-03-08,NaN
9,03/09/2020,16,0,Kings,New York,36047.0,2020-03-09,NaN


In [658]:
K_c = Kings_c.groupby(['fips']).sum().reset_index()
Q_c = Queens_c.groupby(['fips']).sum().reset_index()
B_c = Bronx_c.groupby(['fips']).sum().reset_index()
M_c = Manhattan_c.groupby(['fips']).sum().reset_index()
R_c = Richmond_c.groupby(['fips']).sum().reset_index()
K_c.head(19)

,fips,cases,deaths
0,36047.0,2482,41


In [659]:
covid_death = covid[(covid['dt'] >= data_date_dt- timedelta(days = 1))]
Kings_death = Kings.groupby(by=['fips'])['deaths'].sum().reset_index()
Bronx_death = Bronx.groupby(by=['fips'])['deaths'].sum().reset_index()
Manhattan_death = Manhattan.groupby(by=['fips'])['deaths'].sum().reset_index()
Queens_death = Queens.groupby(by=['fips'])['deaths'].sum().reset_index()
Richmond_death = Richmond.groupby(by=['fips'])['deaths'].sum().reset_index()

In [660]:
# We merge the NY data with the rest of the US data
covid = pd.concat([covid, Kings, Bronx, Manhattan, Queens, Richmond], sort = False)


In [661]:
Queens.tail(14)

,date,cases,deaths,state,county,fips,dt
241,10/27/2020,180,1,New York,Queens,36081.0,2020-10-27
242,10/28/2020,194,0,New York,Queens,36081.0,2020-10-28
243,10/29/2020,154,1,New York,Queens,36081.0,2020-10-29
244,10/30/2020,151,1,New York,Queens,36081.0,2020-10-30
245,10/31/2020,117,2,New York,Queens,36081.0,2020-10-31
246,11/01/2020,96,0,New York,Queens,36081.0,2020-11-01
247,11/02/2020,210,1,New York,Queens,36081.0,2020-11-02
248,11/03/2020,173,2,New York,Queens,36081.0,2020-11-03
249,11/04/2020,261,1,New York,Queens,36081.0,2020-11-04
250,11/05/2020,282,2,New York,Queens,36081.0,2020-11-05


In [662]:

covid.tail(50)

,date,county,state,fips,cases,deaths,dt
205,09/21/2020,Richmond,New York,36085.0,24,0,2020-09-21
206,09/22/2020,Richmond,New York,36085.0,25,0,2020-09-22
207,09/23/2020,Richmond,New York,36085.0,37,1,2020-09-23
208,09/24/2020,Richmond,New York,36085.0,26,0,2020-09-24
209,09/25/2020,Richmond,New York,36085.0,28,0,2020-09-25
210,09/26/2020,Richmond,New York,36085.0,33,0,2020-09-26
211,09/27/2020,Richmond,New York,36085.0,28,0,2020-09-27
212,09/28/2020,Richmond,New York,36085.0,46,0,2020-09-28
213,09/29/2020,Richmond,New York,36085.0,34,0,2020-09-29
214,09/30/2020,Richmond,New York,36085.0,40,0,2020-09-30


In [663]:
print (data_date_dt)

2020-11-13 00:00:00


In [664]:

covid_death = covid_death[['fips', 'deaths']]
covid_death.head(2)

,fips,deaths
722537,1001.0,32
722538,1003.0,84


In [665]:
Kings_death.head(2)

,fips,deaths
0,36047.0,5782


In [666]:
# We merge the NY data with the rest of the US data
covid_death = pd.concat([covid_death, Kings_death, Bronx_death, Manhattan_death, Queens_death, Richmond_death], sort = False)

covid_death = covid_death.rename(columns={"deaths": "total_deaths"})


# Create a dictionary for the cumulative COVID deaths in each county
County_covid_death = dict(zip(covid_death.fips, covid_death.total_deaths))

covid_death.tail(250)

,fips,total_deaths
725536,51115.0,0
725537,51117.0,35
725538,51119.0,11
725539,51121.0,7
725540,51125.0,2
725541,51127.0,3
725542,51700.0,51
725543,51710.0,86
725544,51131.0,31
725545,51133.0,8


In [667]:
covid_death.shape

(3249, 2)

In [668]:
print (County_covid_death)

{1001.0: 32, 1003.0: 84, 1005.0: 9, 1007.0: 17, 1009.0: 34, 1011.0: 19, 1013.0: 41, 1015.0: 77, 1017.0: 48, 1019.0: 15, 1021.0: 36, 1023.0: 12, 1025.0: 18, 1027.0: 13, 1029.0: 11, 1031.0: 12, 1033.0: 37, 1035.0: 14, 1037.0: 3, 1039.0: 32, 1041.0: 30, 1043.0: 30, 1045.0: 55, 1047.0: 28, 1049.0: 34, 1051.0: 60, 1053.0: 31, 1055.0: 60, 1057.0: 15, 1059.0: 33, 1061.0: 8, 1063.0: 17, 1065.0: 30, 1067.0: 6, 1069.0: 36, 1071.0: 23, 1073.0: 454, 1075.0: 7, 1077.0: 49, 1079.0: 35, 1081.0: 65, 1083.0: 38, 1085.0: 29, 1087.0: 20, 1089.0: 117, 1091.0: 24, 1093.0: 33, 1095.0: 53, 1097.0: 334, 1099.0: 10, 1101.0: 221, 1103.0: 41, 1105.0: 6, 1107.0: 18, 1109.0: 14, 1111.0: 19, 1113.0: 3, 1117.0: 73, 1115.0: 54, 1119.0: 21, 1121.0: 40, 1123.0: 87, 1125.0: 147, 1127.0: 96, 1129.0: 12, 1131.0: 12, 1133.0: 21, 2013.0: 0, 2016.0: 0, 2020.0: 57, 2050.0: 1, 2060.0: 0, 2068.0: 0, 2070.0: 0, 2090.0: 17, 2100.0: 0, 2110.0: 2, 2122.0: 3, 2130.0: 0, 2150.0: 0, 2158.0: 0, 2164.0: 0, 2170.0: 5, 2180.0: 0, 2185.0: 

In [669]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [670]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases'] = covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
## set negative new cases to zero, this can occuer due to the disperacy in the data

covid_last15.loc[covid_last15.new_cases < 1e-6, 'new_cases'] = 0
covid_last15.sort_index(inplace = True)

In [671]:
covid_last15[covid_last15.county == 'New Haven'].head(50)

,date,county,state,fips,cases,deaths,dt,dt_time_delta,new_cases
680677,2020-10-30,New Haven,Connecticut,9009.0,17352,1128,2020-10-30,-14 days,NaN
683921,2020-10-31,New Haven,Connecticut,9009.0,17352,1128,2020-10-31,-13 days,0.0
687166,2020-11-01,New Haven,Connecticut,9009.0,17352,1128,2020-11-01,-12 days,0.0
690411,2020-11-02,New Haven,Connecticut,9009.0,18048,1128,2020-11-02,-11 days,696.0
693654,2020-11-03,New Haven,Connecticut,9009.0,18244,1130,2020-11-03,-10 days,196.0
696897,2020-11-04,New Haven,Connecticut,9009.0,18374,1132,2020-11-04,-9 days,130.0
700142,2020-11-05,New Haven,Connecticut,9009.0,18862,1135,2020-11-05,-8 days,488.0
703386,2020-11-06,New Haven,Connecticut,9009.0,19154,1137,2020-11-06,-7 days,292.0
706629,2020-11-07,New Haven,Connecticut,9009.0,19154,1137,2020-11-07,-6 days,0.0
709873,2020-11-08,New Haven,Connecticut,9009.0,19154,1137,2020-11-08,-5 days,0.0


In [672]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()


In [673]:
## group by FIPS to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases'].sum().reset_index(name ='total_cases')

covid_last14_stats[covid_last14_stats.fips == 9009].head(5)

,fips,total_cases
310,9009.0,3885.0


In [674]:
covid.tail(50)

,date,county,state,fips,cases,deaths,dt
205,09/21/2020,Richmond,New York,36085.0,24,0,2020-09-21
206,09/22/2020,Richmond,New York,36085.0,25,0,2020-09-22
207,09/23/2020,Richmond,New York,36085.0,37,1,2020-09-23
208,09/24/2020,Richmond,New York,36085.0,26,0,2020-09-24
209,09/25/2020,Richmond,New York,36085.0,28,0,2020-09-25
210,09/26/2020,Richmond,New York,36085.0,33,0,2020-09-26
211,09/27/2020,Richmond,New York,36085.0,28,0,2020-09-27
212,09/28/2020,Richmond,New York,36085.0,46,0,2020-09-28
213,09/29/2020,Richmond,New York,36085.0,34,0,2020-09-29
214,09/30/2020,Richmond,New York,36085.0,40,0,2020-09-30


In [675]:
## group by FIPS to get case load and follow up demand values for each county
#covid_death = covid.groupby(['fips'])['deaths'].sum().reset_index(name ='total_deaths')
#covid_death.head(250)

In [676]:
## group by FIPS to get case load each month and follow up demand values for each county
covid['month'] = pd.to_datetime(covid['date'])
covid['year'] = pd.to_datetime(covid['date'])


In [677]:
#covid_last14_stats_montly = covid.groupby(['fips', covid.month.dt.month, covid.year.dt.year])['deaths'].sum().reset_index() 

#covid_last14_stats_montly['cumulative_death'] = covid_last14_stats_montly.groupby(['fips'])['deaths'].cumsum(axis = 0) 
#covid_last14_stats_montly = covid_last14_stats_montly.groupby(['fips', covid_last14_stats_montly.month,  covid_last14_stats_montly.year])['deaths'].cumsum()

#covid_last14_stats_montly.head(10)

In [678]:
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'right', right_on = 'FIPS', left_on = 'fips' )
covid_last14_stats.fillna(0 , inplace=True)

In [679]:

covid_last14_stats.tail(5)

,index,fips,total_cases,E_TOTPOP,FIPS,STATE
3215,0.0,0.0,0.0,2132,2105,ALASKA
3216,0.0,0.0,0.0,689,2282,ALASKA
3217,0.0,0.0,0.0,75,15005,HAWAII
3218,0.0,0.0,0.0,981,32009,NEVADA
3219,0.0,0.0,0.0,102,48301,TEXAS


In [680]:
now = pd.to_datetime("now")

print (now)
m_now = now.month
print (m_now)

y_now = now.year

2020-11-13 17:54:50.608437
11


In [681]:
#one_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-1) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#one_month_lag_death = dict(zip(one_month_lag_cumulative_death.fips, one_month_lag_cumulative_death.cumulative_death))

#one_month_lag_cumulative_death.head(5)


In [682]:
#two_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-2) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#two_month_lag_death = dict(zip(two_month_lag_cumulative_death.fips, two_month_lag_cumulative_death.cumulative_death))


#two_month_lag_cumulative_death.head(5)

In [683]:
#three_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now - 3) & (covid_last14_stats_montly['year'] == y_now)]


# Create a dictionary for the 3 last month COVID deaths in each county
#three_month_lag_death = dict(zip(three_month_lag_cumulative_death.fips, three_month_lag_cumulative_death.cumulative_death))

#three_month_lag_cumulative_death.head(5)

#print (three_month_lag_death)

In [684]:

# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = svi_county.FIPS.tolist() #[k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]




In [685]:
for j in County_covid_death:
       
    print (j, County_covid_death[j])

1001.0 32
1003.0 84
1005.0 9
1007.0 17
1009.0 34
1011.0 19
1013.0 41
1015.0 77
1017.0 48
1019.0 15
1021.0 36
1023.0 12
1025.0 18
1027.0 13
1029.0 11
1031.0 12
1033.0 37
1035.0 14
1037.0 3
1039.0 32
1041.0 30
1043.0 30
1045.0 55
1047.0 28
1049.0 34
1051.0 60
1053.0 31
1055.0 60
1057.0 15
1059.0 33
1061.0 8
1063.0 17
1065.0 30
1067.0 6
1069.0 36
1071.0 23
1073.0 454
1075.0 7
1077.0 49
1079.0 35
1081.0 65
1083.0 38
1085.0 29
1087.0 20
1089.0 117
1091.0 24
1093.0 33
1095.0 53
1097.0 334
1099.0 10
1101.0 221
1103.0 41
1105.0 6
1107.0 18
1109.0 14
1111.0 19
1113.0 3
1117.0 73
1115.0 54
1119.0 21
1121.0 40
1123.0 87
1125.0 147
1127.0 96
1129.0 12
1131.0 12
1133.0 21
2013.0 0
2016.0 0
2020.0 57
2050.0 1
2060.0 0
2068.0 0
2070.0 0
2090.0 17
2100.0 0
2110.0 2
2122.0 3
2130.0 0
2150.0 0
2158.0 0
2164.0 0
2170.0 5
2180.0 0
2185.0 0
2188.0 0
2195.0 0
2198.0 1
2220.0 0
2230.0 0
2240.0 1
nan 0
2261.0 1
2275.0 0
2290.0 2
4001.0 187
4003.0 76
4005.0 153
4007.0 77
4009.0 31
4011.0 2
4012.0 18
4013.0 375

21185.0 27
21187.0 3
21189.0 1
21191.0 1
21193.0 9
21195.0 8
21197.0 0
21199.0 13
21201.0 1
21203.0 4
21205.0 2
21207.0 10
21209.0 20
21211.0 25
21213.0 9
21215.0 1
21217.0 7
21219.0 13
21221.0 0
21223.0 0
21225.0 6
21227.0 59
21229.0 3
21231.0 2
21233.0 5
21235.0 8
21237.0 0
21239.0 0
22001.0 108
22003.0 64
22005.0 100
22007.0 25
22009.0 65
22011.0 43
22013.0 35
22015.0 127
22017.0 417
22019.0 233
22021.0 4
22023.0 3
22025.0 15
22027.0 27
22029.0 17
22031.0 35
22033.0 463
22035.0 6
22037.0 83
22039.0 40
22041.0 53
22043.0 31
22045.0 99
22047.0 62
22049.0 24
22051.0 593
22053.0 45
22059.0 12
22055.0 141
22057.0 131
22061.0 53
22063.0 83
22065.0 11
22067.0 29
22069.0 35
22071.0 605
22073.0 174
22075.0 19
22077.0 44
22079.0 169
22081.0 25
22083.0 21
22085.0 16
22087.0 32
22089.0 65
22091.0 4
22093.0 40
22095.0 108
22097.0 140
22099.0 65
22101.0 87
22103.0 277
22105.0 130
22107.0 3
22109.0 126
22111.0 54
nan 234
22113.0 70
22115.0 57
22117.0 73
22119.0 45
22121.0 42
22123.0 14
22125.0 25


40067.0 1
40069.0 4
40071.0 18
40073.0 6
40075.0 3
40077.0 3
40079.0 25
40081.0 23
40083.0 3
40085.0 1
40093.0 2
40095.0 2
40097.0 17
40087.0 16
40089.0 41
40091.0 12
40099.0 3
40101.0 29
40103.0 3
40105.0 4
40107.0 11
40109.0 254
40111.0 14
40113.0 17
40115.0 21
40117.0 5
40119.0 15
40121.0 21
40123.0 8
40125.0 21
40127.0 6
40129.0 5
40131.0 57
40133.0 7
40135.0 14
40137.0 10
40139.0 11
40141.0 4
40143.0 231
nan 0
40145.0 32
40147.0 43
40149.0 2
40151.0 1
40153.0 6
41001.0 3
41003.0 6
41005.0 72
41007.0 0
41009.0 2
41011.0 1
41013.0 5
41015.0 2
41017.0 14
41019.0 10
41021.0 0
41023.0 0
41025.0 0
41027.0 1
41029.0 8
41031.0 11
41033.0 4
41035.0 3
41037.0 0
41039.0 33
41041.0 13
41043.0 17
41045.0 38
41047.0 122
41049.0 7
41051.0 191
41053.0 15
41055.0 0
41057.0 0
41059.0 45
41061.0 2
41063.0 2
41065.0 17
41067.0 89
41069.0 0
41071.0 15
42001.0 36
42003.0 456
42005.0 29
42007.0 142
42009.0 11
42011.0 438
42013.0 41
42015.0 22
42017.0 636
42019.0 40
42021.0 15
42023.0 0
42025.0 35
42027.

In [686]:
K_c.head()

,fips,cases,deaths
0,36047.0,2482,41


In [687]:
K = dict(zip(K_c.fips, K_c.cases))
Q = dict(zip(Q_c.fips, Q_c.cases))
B = dict(zip(B_c.fips, B_c.cases))
M = dict(zip(M_c.fips, M_c.cases))
R  = dict(zip(R_c.fips, R_c.cases))

In [688]:
print (K)

{36047.0: 2482}


In [689]:
# Create a dictionary for the county and covid cases
covid_cases_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.total_cases))
COVID_14days = {}

for j in location:
    if j in covid_cases_county_ny_times:
        COVID_14days[j] = covid_cases_county_ny_times[j] 
        print(j, covid_cases_county_ny_times[j])
        

        
    else:
        COVID_14days[j] = 0
        
        
    if j not in County_covid_death:
        print ('j',j)
        County_covid_death[j] = 0

#print (three_month_lag_death)
#print (two_month_lag_covid_death)
#print (one_month_lag_covid_death)
#for j in location:
#    if j not in three_month_lag_death:        
#        three_month_lag_death[j] = 0
        
#    if j not in two_month_lag_death:        
#        two_month_lag_death[j] = 0
        
#    if j not in one_month_lag_death:        
#        one_month_lag_death[j] = 0
        
    #if j not in County_covid_death:
    #    County_covid_death[j] = 0

35039 201.0
1001 244.0
1009 390.0
1013 42.0
1015 539.0
1017 133.0
1031 223.0
1033 478.0
1039 204.0
1043 735.0
1045 249.0
1051 252.0
1055 642.0
1067 63.0
1069 366.0
1071 313.0
1077 703.0
1079 138.0
1083 509.0
1089 1187.0
1095 566.0
1097 1062.0
1103 729.0
1111 59.0
1113 36.0
1115 475.0
1117 975.0
1121 268.0
2261 37.0
4021 1216.0
5009 232.0
5011 17.0
5033 400.0
5037 56.0
5045 476.0
5047 93.0
5051 448.0
5053 59.0
5063 315.0
5083 94.0
5085 340.0
5087 73.0
5115 274.0
5117 69.0
5121 125.0
5125 683.0
5131 915.0
5145 571.0
6007 227.0
6017 194.0
6023 82.0
6027 18.0
6061 703.0
6079 674.0
6089 770.0
6093 108.0
8019 34.0
8039 115.0
8047 22.0
8051 81.0
8057 2.0
8065 40.0
8067 169.0
8071 21.0
8075 355.0
8077 1495.0
8083 101.0
8085 105.0
8093 40.0
8099 124.0
8101 1755.0
8103 19.0
8119 117.0
8121 37.0
9005 601.0
9007 476.0
9011 715.0
9015 307.0
10001 356.0
12019 598.0
12033 1164.0
12089 193.0
12101 1276.0
12109 740.0
12113 455.0
13015 450.0
13039 128.0
13045 674.0
13047 212.0
13057 1034.0
13073 664.0
1

51073 23.0
51075 35.0
51077 48.0
51079 27.0
51083 109.0
51085 258.0
51089 178.0
51093 38.0
51095 60.0
51097 12.0
51099 37.0
51101 34.0
51109 26.0
51113 27.0
51127 34.0
51137 63.0
51139 38.0
51143 132.0
51155 90.0
51165 159.0
51171 151.0
51177 209.0
51181 8.0
51187 65.0
51191 342.0
51193 27.0
51197 80.0
51199 69.0
51550 454.0
51580 48.0
51735 14.0
51800 148.0
51810 907.0
51820 36.0
53003 153.0
53011 1246.0
53015 145.0
53023 23.0
53027 108.0
53029 71.0
53035 298.0
53037 21.0
53041 120.0
53045 64.0
53059 7.0
53065 108.0
53067 417.0
54003 412.0
54009 87.0
54023 25.0
54025 76.0
54029 96.0
54031 29.0
54033 143.0
54035 76.0
54037 208.0
54041 81.0
54049 133.0
54051 301.0
54053 39.0
54057 231.0
54065 40.0
54067 49.0
54073 17.0
54079 334.0
54083 96.0
54085 34.0
54091 29.0
54095 52.0
54097 68.0
54103 87.0
54107 638.0
55003 175.0
55005 916.0
55009 3286.0
55011 202.0
55015 658.0
55017 1362.0
55021 705.0
55023 223.0
55027 1855.0
55031 360.0
55035 1781.0
55039 1969.0
55041 142.0
55043 749.0
55045 305

j 2105
2150 41.0
2170 735.0
2198 6.0
2240 42.0
4007 294.0
4013 13114.0
4015 498.0
5049 79.0
5057 120.0
5065 243.0
5067 324.0
5089 77.0
5097 22.0
5101 38.0
5111 265.0
5127 59.0
5137 18.0
5147 32.0
6003 8.0
6021 49.0
6045 104.0
6065 5491.0
6069 102.0
6073 5965.0
6095 962.0
6113 383.0
8011 11.0
8031 6900.0
8089 81.0
8105 23.0
9003 3332.0
11001 1363.0
12011 6566.0
12063 326.0
12071 1940.0
12095 3711.0
12097 1303.0
12133 76.0
13001 37.0
13009 92.0
13025 52.0
13027 15.0
13033 68.0
13055 121.0
13059 527.0
13087 75.0
13089 2053.0
13091 121.0
13097 479.0
13101 5.0
13121 2547.0
13125 14.0
13151 749.0
13155 19.0
13173 15.0
13175 170.0
13189 98.0
13197 12.0
13225 83.0
13245 832.0
13249 12.0
13279 54.0
13287 23.0
13315 18.0
15009 55.0
16003 20.0
16047 189.0
16063 64.0
17003 49.0
17019 1973.0
17151 15.0
17153 59.0
18001 466.0
18027 210.0
18087 270.0
18105 626.0
19021 312.0
19051 99.0
20067 100.0
20075 11.0
21011 51.0
21041 27.0
21047 341.0
21099 138.0
21115 181.0
21143 30.0
21165 16.0
21169 81.0
211

In [690]:
for j in location: 
    if j in K:
        COVID_14days[j] = K[j]
        print (j, COVID_14days[j], K[j])

    if j in Q:
        COVID_14days[j] = Q[j]
        print (j, COVID_14days[j])
        
    if j in B:
        COVID_14days[j] = B[j]
        print (j, COVID_14days[j])
        
    if j in M:
        COVID_14days[j] = M[j]
        print (j, COVID_14days[j])

    if j in R:
        COVID_14days[j] = R[j]
        print (j, COVID_14days[j])

36085 904
36061 1384
36047 2482 2482
36081 2212
36005 1343


In [691]:
print (covid_cases_county_ny_times)

{1001.0: 244.0, 1003.0: 566.0, 1005.0: 57.0, 1007.0: 100.0, 1009.0: 390.0, 1011.0: 18.0, 1013.0: 42.0, 1015.0: 539.0, 1017.0: 133.0, 1019.0: 101.0, 1021.0: 122.0, 1023.0: 12.0, 1025.0: 82.0, 1027.0: 59.0, 1029.0: 66.0, 1031.0: 223.0, 1033.0: 478.0, 1035.0: 54.0, 1037.0: 28.0, 1039.0: 204.0, 1041.0: 49.0, 1043.0: 735.0, 1045.0: 249.0, 1047.0: 131.0, 1049.0: 478.0, 1051.0: 252.0, 1053.0: 87.0, 1055.0: 642.0, 1057.0: 98.0, 1059.0: 179.0, 1061.0: 106.0, 1063.0: 19.0, 1065.0: 64.0, 1067.0: 63.0, 1069.0: 366.0, 1071.0: 313.0, 1073.0: 2725.0, 1075.0: 53.0, 1077.0: 703.0, 1079.0: 138.0, 1081.0: 374.0, 1083.0: 509.0, 1085.0: 30.0, 1087.0: 80.0, 1089.0: 1187.0, 1091.0: 65.0, 1093.0: 125.0, 1095.0: 566.0, 1097.0: 1062.0, 1099.0: 50.0, 1101.0: 799.0, 1103.0: 729.0, 1105.0: 29.0, 1107.0: 140.0, 1109.0: 91.0, 1111.0: 59.0, 1113.0: 36.0, 1115.0: 475.0, 1117.0: 975.0, 1119.0: 30.0, 1121.0: 268.0, 1123.0: 157.0, 1125.0: 799.0, 1127.0: 301.0, 1129.0: 93.0, 1131.0: 25.0, 1133.0: 129.0, 2013.0: 4.0, 2016.

In [692]:
#Parameters
pro_c_s = [(i,county_of_states[i]) for i in location ]
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [693]:
df = covid_last14_stats
df['fips'] = df['fips'].astype(int)

In [694]:
###############################################################################################
######################## END calculating different types of vulnerabilities ###################

Since we allocate CHW proportional to the county values of certain vulnaribilites within state, we need a few function to help us with the calculations. 



In [695]:
# This function return the value for the state for the given dictionary

# More specifically sum upt the values for the counties of each state

def total_state(dict_1):
    state_dict = {}
    for s in State:
        state_dict [s] = sum(float(dict_1[j]) for (j,s) in cartesian_pro_county_state.select('*', s) if j in dict_1)  
    return state_dict



In [696]:
# Calculte the population per state by summing up the population in each county in the state
State_pop = total_state(population_county)

In [697]:


# This function returns the ratio of the dict value for county and state of the county
def Proportional(county_level, state_level):
    
    prop = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            prop[j] = (float(county_level[j])/float(state_level[s]))    
        else:
            prop[j] = 0
                
    return prop

In [698]:
print (ACI_total)
print (ACI_total[35039])

{1001: 9049, 1003: 30763, 1005: 7244, 1007: 4272, 1009: 9290, 1011: 2935, 1013: 4814, 1015: 25890, 1017: 7400, 1019: 5105, 1021: 9147, 1023: 3273, 1025: 6716, 1027: 3112, 1029: 3180, 1031: 9926, 1033: 9540, 1035: 2765, 1037: 2155, 1039: 6835, 1041: 3219, 1043: 15478, 1045: 10930, 1047: 13544, 1049: 17896, 1051: 12373, 1053: 6235, 1055: 21998, 1057: 4144, 1059: 6715, 1061: 6647, 1063: 3143, 1065: 4730, 1067: 3651, 1069: 22918, 1071: 11352, 1073: 126944, 1075: 3099, 1077: 15561, 1079: 6877, 1081: 23596, 1083: 14596, 1085: 3540, 1087: 5301, 1089: 48769, 1091: 5053, 1093: 6051, 1095: 20669, 1097: 90962, 1099: 5565, 1101: 52763, 1103: 22180, 1105: 3375, 1107: 4807, 1109: 6945, 1111: 4428, 1113: 14480, 1115: 12599, 1117: 20035, 1119: 4098, 1121: 19979, 1123: 9968, 1125: 34877, 1127: 13887, 1129: 4268, 1131: 4086, 1133: 5577, 2013: 444, 2016: 400, 2020: 49663, 2050: 8680, 2060: 141, 2068: 142, 2070: 1983, 2090: 12661, 2100: 606, 2105: 504, 2110: 4631, 2122: 11347, 2130: 2881, 2150: 2412, 2158

In [699]:
#ACI
ACI_State = total_state(ACI_total)

In [700]:
# Create dicts for the variables of SVI
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.EP_UNEMP))
E_PCI = dict(zip(svi_county.FIPS, svi_county.EP_PCI))
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.EP_NOHSDP))
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.EP_AGE65))
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.EP_AGE17))
E_DISABL = dict(zip(svi_county.FIPS, svi_county.EP_DISABL))
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.EP_SNGPNT))
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.EP_MINRTY))
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.EP_LIMENG))
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.EP_MUNIT))
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.EP_MOBILE))
E_CROWD = dict(zip(svi_county.FIPS, svi_county.EP_CROWD))
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.EP_NOVEH))
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.EP_GROUPQ))


# Calculate the state value for the SVI variables
E_POV_State = total_state(E_POV)
E_UNEMP_State = total_state(E_UNEMP) 
E_PCI_State = total_state(E_PCI)
E_NOHSDP_State = total_state(E_NOHSDP) 
E_AGE65_State = total_state(E_AGE65)
E_AGE17_State = total_state(E_AGE17)
E_DISABL_State = total_state(E_DISABL)
E_SNGPNT_State = total_state(E_SNGPNT)
E_MINRTY_State = total_state(E_MINRTY)
E_LIMENG_State = total_state(E_LIMENG)
E_MUNIT_State = total_state(E_MUNIT)
E_MOBILE_State = total_state(E_MOBILE)
E_CROWD_State = total_state(E_CROWD)
E_NOVEH_State = total_state(E_NOVEH)
E_GROUPQ_State = total_state(E_GROUPQ)



# Calculate the proportinal values for the SVI variables
E_POV_Prop = Proportional(E_POV, E_POV_State )
E_UNEMP_Prop = Proportional(E_UNEMP, E_UNEMP_State ) 
E_PCI_Prop = Proportional(E_PCI, E_PCI_State )
E_NOHSDP_Prop = Proportional(E_NOHSDP, E_NOHSDP_State ) 
E_AGE65_Prop = Proportional(E_AGE65, E_AGE65_State )
E_AGE17_Prop = Proportional(E_AGE17, E_AGE17_State )
E_DISABL_Prop = Proportional(E_DISABL, E_DISABL_State )
E_SNGPNT_Prop = Proportional(E_SNGPNT, E_SNGPNT_State )
E_MINRTY_Prop = Proportional(E_MINRTY, E_MINRTY_State )
E_LIMENG_Prop = Proportional(E_LIMENG, E_LIMENG_State )
E_MUNIT_Prop = Proportional(E_MUNIT, E_MUNIT_State )
E_MOBILE_Prop = Proportional(E_MOBILE, E_MOBILE_State )
E_CROWD_Prop = Proportional(E_CROWD, E_CROWD_State )
E_NOVEH_Prop = Proportional(E_NOVEH, E_NOVEH_State )
E_GROUPQ_Prop = Proportional(E_GROUPQ, E_GROUPQ_State )

ACI_Prop = Proportional(ACI_total, ACI_State)


# SVI calculation 

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Minority, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We will use these variables in a county base and state base. While County base values are exactly same as the estimated values for these variables in the CDC website, to calculate the state base, we simply sum the county values for all of the counties in each state for each variable. Then we use the following formula to calculate the SVI value for each county.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{15}\sum_{k \in K} \frac{c^k_j}{c^k_s}$


In [701]:

from collections import Counter
# Sum all SVI variable values for each county
SVI_county_sum = dict(Counter(E_POV_Prop) + Counter(E_UNEMP_Prop) + Counter(E_PCI_Prop) + Counter(E_NOHSDP_Prop) + Counter(E_AGE65_Prop) + Counter(E_AGE17_Prop) + Counter(E_DISABL_Prop) + Counter(E_SNGPNT_Prop) + Counter(E_MINRTY_Prop) + Counter(E_LIMENG_Prop) + Counter(E_MUNIT_Prop) + Counter(E_MOBILE_Prop) + Counter(E_CROWD_Prop) + Counter(E_NOVEH_Prop) + Counter(E_GROUPQ_Prop))

# Divide the sum of all SVI variable values
SVI_county = {j: SVI_county_sum[j]/15 for j in SVI_county_sum }


# Proportional Allocation

We consider allocating 1 million CHW over the states proportional to Medicaid enrollment in each state. Further, we allocate CHW to counties in each state proportional to different county vulnerability criterias as follow.

- MEDICAID
- SVI
- YPLL
- UNEMPLOYMENT
- LAST 14 DAYS COVID CASES
- LAST 14 DAYS COVID CASES / POP
- COVID DEATHS / POP

To calculate the total number of allocated CHW to per county according to these vulnerability criterias, we define the following function called "Proportional_allocation", in which we multiply the CHW allocated to each state with the ratio of the chosen vulnerability criteria of the county to the chosen vulnerability criteria of the state, the function return a dictionary with the counties as keys and the number of CHW allocated to each county for the chosen vulnerability criteria as values. 

In [702]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            #print (j,s, county_level[j],state_level[s], state_budget[s])
            prop_allocate[j,s] = (float(county_level[j])/float(state_level[s]))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

In [703]:
Medicaid_dem = Proportional_allocation(ACI_total, ACI_State, Medicaid_state)
Medicaid_demand = {m[0]: Medicaid_dem[m] for m in Medicaid_dem}
for s in State:
#    print (s)
#    print ( ACI_State[s])
    print (s,Medicaid_state[s])
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

NEW MEXICO 772102
ALABAMA 957116
ALASKA 231145
ARIZONA 1839932
ARKANSAS 830467
CALIFORNIA 11847711
COLORADO 1337805
CONNECTICUT 874974
DELAWARE 239009
FLORIDA 3892552
GEORGIA 1928703
IDAHO 340742
ILLINOIS 2987496
INDIANA 1602976
IOWA 699741
KANSAS 401103
KENTUCKY 1416013
LOUISIANA 1585024
MAINE 232455
MARYLAND 1372695
MASSACHUSETTS 1616404
MICHIGAN 2439425
MINNESOTA 1085778
MISSISSIPPI 632427
MISSOURI 923641
MONTANA 247333
NEBRASKA 254159
NEVADA 685073
NEW HAMPSHIRE 193436
NEW JERSEY 1759653
NEW YORK 6263164
NORTH CAROLINA 1851558
NORTH DAKOTA 96757
OHIO 2788134
OKLAHOMA 797220
OREGON 1053931
PENNSYLVANIA 3069309
RHODE ISLAND 305208
SOUTH CAROLINA 1048276
SOUTH DAKOTA 114059
TENNESSEE 1489536
TEXAS 4457644
UTAH 338812
VERMONT 161049
VIRGINIA 1497770
WASHINGTON 1780968
WEST VIRGINIA 521290
WISCONSIN 1112844
WYOMING 59302
HAWAII 351337
DISTRICT OF COLUMBIA 248591
PUERTO RICO 1622194


In [704]:
#print ('Med', Medicaid_demand[35039])
#print ('Med', Medicaid_demand[4017], 'Pop', population_county[4017], 'Med_capita', 100000*(Medicaid_demand[4017]/population_county[4017]))

In [705]:
#Further we create additional vulnerability values by considering SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita and the number of Medicaid enrolles in each county together

Covid_capita = {j: 100000*(COVID_14days[j]/population_county[j]) for j in location}

Covid_death_capita = {j: 100000*(County_covid_death[j]/population_county[j]) for j in location}

Medicaid_capita = {j: 100000*(Medicaid_demand[j]/population_county[j]) for j in location}

Unemployment_capita = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Rate))

#Three_month_lag_covid_death_capita = {j: 100000*(three_month_lag_death[j]/population_county[j]) for j in location}

#Two_month_lag_covid_death_capita = {j: 100000*(two_month_lag_death[j]/population_county[j]) for j in location}

#One_month_lag_covid_death_capita = {j: 100000*(one_month_lag_death[j]/population_county[j]) for j in location}

In [706]:
for j in location:
    print (j, County_covid_death[j], population_county[j], Covid_death_capita[j])

35039 18 38921 46.24752704195678
1001 32 55869 57.27684404589307
1009 34 57826 58.79708089786602
1013 41 19448 210.81859317153436
1015 77 113605 67.77870692311079
1017 48 33254 144.3435376195345
1031 12 52342 22.926139620190288
1033 37 55241 66.97923643670462
1039 32 37049 86.372101811115
1043 30 83768 35.813198357367966
1045 55 49172 111.85227365167168
1051 60 81209 73.88343656491276
1055 60 102268 58.66937849571713
1067 6 17205 34.873583260680036
1069 36 105882 34.00011333371111
1071 23 51626 44.55119513423469
1077 49 92729 52.842152940288365
1079 35 32924 106.30543068885919
1083 38 98915 38.41682252438963
1089 117 372909 31.374946702814896
1095 53 96774 54.76677620021907
1097 334 413210 80.8305704121391
1103 41 119679 34.258307639602606
1111 19 22722 83.61939970073057
1113 3 57961 5.175894135711944
1115 54 89512 60.32710698006971
1117 73 217702 33.532075957042196
1121 40 79978 50.01375378229013
2261 1 9202 10.867202782003913
4021 234 462789 50.56299955271192
5009 40 37432 106.860440

27167 4 6207 64.4433703882713
27173 8 9709 82.39777526006797
28003 21 36953 56.82894487592347
28033 85 184945 45.959609613668924
28045 34 47632 71.38058448102117
28057 32 23390 136.81060282171867
28059 103 143617 71.7185291434858
28081 87 85436 101.83061004728685
28089 97 106272 91.27521830773863
28093 38 35294 107.66702555675185
28095 76 35252 215.5906047883808
28109 64 55535 115.24263977671738
28121 89 155271 57.31913879604047
28129 16 15916 100.52777079668257
28159 22 17955 122.52854358117517
28161 24 12108 198.21605550049554
29003 8 17712 45.16711833785004
29007 11 25388 43.327556325823224
29011 4 11754 34.03096818104475
29013 8 16172 49.46821667078902
29021 54 87364 61.810356668650705
29025 1 9020 11.086474501108647
29031 60 78871 76.0735885179597
29033 4 8679 46.08825901601567
29037 35 105780 33.087540177727355
29039 4 14349 27.87650707366367
29041 0 7426 0.0
29043 20 88595 22.574637394886846
29045 2 6797 29.424746211563924
29047 63 249948 25.20524269047962
29049 38 20387 186.393

55123 4 30822 12.977743170462656
55127 42 103868 40.43593792120769
55131 57 136034 41.901289383536465
55135 87 50990 170.62169052755442
55137 6 24443 24.546905044388986
55141 18 72999 24.657872025644185
56003 9 11790 76.33587786259542
56007 4 14800 27.027027027027028
56009 5 13822 36.17421501953408
56011 1 7584 13.185654008438817
56013 21 39261 53.488194391380766
56015 6 13211 45.41669820604042
56019 2 8445 23.682652457075193
56021 14 99500 14.07035175879397
56025 16 79858 20.03556312454607
56029 3 29194 10.276084126875386
56031 4 8393 47.65876325509353
56033 8 30485 26.242414302115794
56035 1 9831 10.171905197843556
56043 7 7805 89.68609865470852
56045 0 6927 0.0
1003 84 223234 37.62867663527957
1019 15 26196 57.26065048098946
1021 36 44428 81.02998109300441
1027 13 13235 98.22440498677749
1041 30 13772 217.83328492593665
1049 34 71513 47.54380322458854
1057 15 16302 92.01324990798675
1061 8 26271 30.45182901298009
1073 454 658573 68.93692878390095
1075 7 13805 50.70626584570808
1081 

49003 8 56046 14.273989223138138
49005 12 128289 9.353880691251783
49007 3 20463 14.660606949127693
49011 51 355481 14.346758335888557
49013 2 19938 10.031096398836393
49015 3 10012 29.96404314822213
49019 1 9754 10.25220422390814
49021 4 54839 7.294079031346304
49027 1 13188 7.582650894752806
49029 0 12124 0.0
49033 0 2483 0.0
49041 2 21620 9.250693802035153
49043 1 42145 2.372760707082691
49045 8 72259 11.071285237825045
49047 1 35734 2.7984552527005095
49051 10 34091 29.3332551113197
49057 38 260213 14.603421043529725
50001 2 36777 5.4381814721157244
50007 39 163774 23.81330369899984
50009 0 6163 0.0
50021 1 58191 1.7184788025639703
51001 21 32316 64.98329001114
51003 25 109330 22.8665508094759
51036 5 6963 71.80812868016659
51037 2 11880 16.835016835016837
51087 242 330818 73.15200502995604
51091 0 2190 0.0
51103 1 10603 9.431293030274452
51115 0 8834 0.0
51117 35 30587 114.4276980416517
51119 11 10582 103.95010395010395
51125 2 14930 13.395847287340922
51131 31 11710 264.730999146

29023 8 42478 18.833278402937992
29029 41 46305 88.5433538494763
29063 8 12547 63.76026141707181
29149 0 10529 0.0
29185 1 9397 10.641694157709908
29197 0 4660 0.0
29199 1 4902 20.39983680130559
29229 11 18289 60.14544261577998
30023 9 9140 98.46827133479212
30041 21 16484 127.39626304295075
30053 6 19980 30.03003003003003
30061 0 4397 0.0
30101 8 4736 168.91891891891893
31005 0 463 0.0
31133 0 2613 0.0
31153 46 187196 24.573174640483767
32013 4 16831 23.76567048897867
32017 1 5183 19.29384526336099
32029 0 4123 0.0
32031 212 471519 44.9610726184947
34003 2078 932202 222.91305961583433
35001 249 679121 36.665042017549155
35015 30 58460 51.31713992473486
35017 5 26998 18.519890362249054
35021 0 625 0.0
35035 20 67490 29.6340198547933
35037 2 8253 24.23361201987156
35049 21 150358 13.966666223280436
35061 11 76688 14.343834759023578
36001 133 305506 43.53433320458518
36041 1 4416 22.644927536231883
36067 193 460528 41.908418163499285
37041 18 13943 129.09703794018503
37051 108 335509 32.

28031 31 18636 166.34470916505688
28041 19 13586 139.8498454291182
28065 15 11128 134.79511143062547
28097 23 9775 235.2941176470588
28099 111 29118 381.2075005151453
28113 58 39288 147.62777438403586
28115 24 32174 74.59439298812705
29119 16 22837 70.06174191005825
29215 6 25398 23.623907394283016
29221 21 24730 84.91710473109583
29510 230 300576 76.51974874906846
31037 7 10709 65.36558035297413
32009 0 873 0.0
32023 18 46523 38.69054016293016
32027 0 6725 0.0
34001 270 263670 102.40072818295596
35041 10 18500 54.054054054054056
36061 2548 1628706 156.44321320115478
37005 1 11137 8.979078746520607
37007 13 24446 53.1784340996482
37065 66 51472 128.22505439850792
37093 28 55234 50.69341347720607
37107 45 55949 80.43039196411017
37163 31 63531 48.79507641938581
37181 47 44535 105.5349724935444
37187 8 11580 69.08462867012089
39075 29 43960 65.9690627843494
40091 12 19596 61.2369871402327
41045 38 30571 124.3008079552517
45025 51 45650 111.71960569550932
45027 69 33745 204.47473699807378

In [707]:
County_covid_death[9009], population_county[9009], Covid_death_capita[9009]

(1155, 854757, 135.12612356494301)

In [708]:
# Dictionary for the total Covid per capita for each state
Total_covid_cap = total_state(Covid_capita) 


# Dictionary for the total Covid death per capita for each state
Total_covid_death_cap = total_state(Covid_death_capita) 


# Dictionary for the total Medicaid per capita for each state
Total_medicaid_cap = total_state(Medicaid_capita)


# Dictionary for the total Unemployment per capita for each state
Total_unemployment_cap = total_state(Unemployment_capita)



In [709]:
# Dictionaries for the different vulnerability criteria values for states

# Dictionary for total Medicaid patient numbers for each state
Medicaid_demand_state = total_state(Medicaid_demand) 

# Dictionary for total positive COVID cases for last 14 days in each state
Covid_state = total_state(COVID_14days) 

# Dictionary for total SVI values for each state
SVI_state = total_state(SVI_county) 

# Dictionary for total YPLL values for each state
YPLL_state = total_state(YPLL) 

# Dictionary for total Unemployment numbers for each state
Unemployment_state = total_state(Unemployment)  

In [710]:
print(Medicaid_demand_state)

{'NEW MEXICO': 772102.0, 'ALABAMA': 957115.9999999999, 'ALASKA': 231145.00000000006, 'ARIZONA': 1839932.0, 'ARKANSAS': 830466.9999999999, 'CALIFORNIA': 11847710.999999998, 'COLORADO': 1337804.9999999998, 'CONNECTICUT': 874974.0, 'DELAWARE': 239009.0, 'FLORIDA': 3892552.0, 'GEORGIA': 1928703.000000001, 'IDAHO': 340741.9999999999, 'ILLINOIS': 2987495.999999999, 'INDIANA': 1602975.9999999995, 'IOWA': 699741.0000000003, 'KANSAS': 401103.00000000006, 'KENTUCKY': 1416012.9999999995, 'LOUISIANA': 1585023.9999999998, 'MAINE': 232455.00000000003, 'MARYLAND': 1372695.0, 'MASSACHUSETTS': 1616404.0, 'MICHIGAN': 2439425.0000000005, 'MINNESOTA': 1085777.9999999998, 'MISSISSIPPI': 632427.0000000002, 'MISSOURI': 923641.0000000002, 'MONTANA': 247333.0, 'NEBRASKA': 254158.99999999997, 'NEVADA': 685073.0000000001, 'NEW HAMPSHIRE': 193436.0, 'NEW JERSEY': 1759653.0000000002, 'NEW YORK': 6263164.0, 'NORTH CAROLINA': 1851558.0000000005, 'NORTH DAKOTA': 96757.00000000001, 'OHIO': 2788134.0, 'OKLAHOMA': 79722

# 1 million CHW allocation to states

We allocate 1 million CHWs to states proportional to total Medicaid enrolles in each state.

Let's $FedCHW$ represents the number of CHW will be allocated within states by the federal government, which is 1 million in our project. $TotMed$ represents the total Medicaid enrollee numbers over the US, $Med_s$ is the total Medicaid enrollee numbers in state $s \in S$, and $CHW_s$ is the total number of CHW allocated to state $ s\in S$. 

$CHW_s = FedCHW*\frac{Med_s}{TotMed}$



In [711]:
# We consider allocation of 1 million CHW all over the US
Federal_budget_CHW = 1000000

# First, we calculate the Total Medicaid enrolles all over the US
Total_federal_need = sum(Medicaid_demand_state[s] for s in State)

# Allocate the 1 million CHWs proportional to Medicaid enrolles in each state
Medicaid_budget_state = {s: (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW  for s in State}

In [712]:
for s in State:
    print (s, Medicaid_demand_state[s], Total_federal_need, Federal_budget_CHW, (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW)

NEW MEXICO 772102.0 76256043.0 1000000 10125.12542776446
ALABAMA 957115.9999999999 76256043.0 1000000 12551.3462585516
ALASKA 231145.00000000006 76256043.0 1000000 3031.1696084204114
ARIZONA 1839932.0 76256043.0 1000000 24128.343507149984
ARKANSAS 830466.9999999999 76256043.0 1000000 10890.5073923125
CALIFORNIA 11847710.999999998 76256043.0 1000000 155367.50313676777
COLORADO 1337804.9999999998 76256043.0 1000000 17543.593233653624
CONNECTICUT 874974.0 76256043.0 1000000 11474.159497103725
DELAWARE 239009.0 76256043.0 1000000 3134.295861640762
FLORIDA 3892552.0 76256043.0 1000000 51045.81678857897
GEORGIA 1928703.000000001 76256043.0 1000000 25292.461084035018
IDAHO 340741.9999999999 76256043.0 1000000 4468.393409818024
ILLINOIS 2987495.999999999 76256043.0 1000000 39177.17052273483
INDIANA 1602975.9999999995 76256043.0 1000000 21020.97010200227
IOWA 699741.0000000003 76256043.0 1000000 9176.203910816621
KANSAS 401103.00000000006 76256043.0 1000000 5259.950349115283
KENTUCKY 1416012.99

In [713]:
print (Covid_state)

{'NEW MEXICO': 14868.0, 'ALABAMA': 20151.0, 'ALASKA': 5854.0, 'ARIZONA': 22545.0, 'ARKANSAS': 17243.0, 'CALIFORNIA': 77365.0, 'COLORADO': 44343.0, 'CONNECTICUT': 14514.0, 'DELAWARE': 2793.0, 'FLORIDA': 63143.0, 'GEORGIA': 31533.0, 'IDAHO': 14703.0, 'ILLINOIS': 133787.0, 'INDIANA': 55175.0, 'IOWA': 47007.0, 'KANSAS': 24741.0, 'KENTUCKY': 25221.0, 'LOUISIANA': 9758.0, 'MAINE': 1830.0, 'MARYLAND': 15512.0, 'MASSACHUSETTS': 21625.0, 'MICHIGAN': 66078.0, 'MINNESOTA': 56214.0, 'MISSISSIPPI': 11335.0, 'MISSOURI': 43810.0, 'MONTANA': 11217.0, 'NEBRASKA': 22768.0, 'NEVADA': 15334.0, 'NEW HAMPSHIRE': 2500.0, 'NEW JERSEY': 30351.0, 'NEW YORK': 34190.0, 'NORTH CAROLINA': 31758.0, 'NORTH DAKOTA': 16694.0, 'OHIO': 61675.0, 'OKLAHOMA': 23187.0, 'OREGON': 9496.0, 'PENNSYLVANIA': 43581.0, 'RHODE ISLAND': 6079.0, 'SOUTH CAROLINA': 14924.0, 'SOUTH DAKOTA': 16169.0, 'TENNESSEE': 36197.0, 'TEXAS': 115091.0, 'UTAH': 30664.0, 'VERMONT': 496.0, 'VIRGINIA': 18443.0, 'WASHINGTON': 17782.0, 'WEST VIRGINIA': 6929

# Proportional allocation for different vulnerability values

Let V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}. We assume $v_j$ represent the vulnerability value for county $j \in J$, while $v_s$ represent the sum of the vulnerability values for each county in the state of county j.

$Prop_{v_j} = \frac{v_j}{v_s}*CHW_s$



In [714]:
# Calling proportional allocation function for different vulnerability criterias

# Proportional allocation according to cumulative Covid death in per capita in each county
Proportional_to_covid_death_cap = Proportional_allocation(Covid_death_capita, Total_covid_death_cap,Medicaid_budget_state)

# Propportional allocation according to Medicaid enrollee number in each county
Proportional_to_medicaid = Proportional_allocation(Medicaid_demand, Medicaid_demand_state,Medicaid_budget_state )

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_medicaid_cap = Proportional_allocation(Medicaid_capita, Total_medicaid_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases in each county
Proportional_to_covid = Proportional_allocation(COVID_14days, Covid_state, Medicaid_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_SVI = Proportional_allocation(SVI_county, SVI_state, Medicaid_budget_state)

# Proportional allocation according to YPLL in each county
Proportional_to_YPLL = Proportional_allocation(YPLL, YPLL_state, Medicaid_budget_state)

# Proportional allocation according to Unemployment  in each county
Proportional_to_unemployment = Proportional_allocation(Unemployment, Unemployment_state, Medicaid_budget_state)

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_unemployment_cap = Proportional_allocation(Unemployment_capita, Total_unemployment_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases per capita in each county
Proportional_to_covid_capita = Proportional_allocation(Covid_capita, Total_covid_cap, Medicaid_budget_state)



# Normalize values for comparison
To be able compare the different vulnerability values for each county, we normalize all vulnerability values as follows. 

Let $m_{v_s} = \min \{v_j, \text{ for county j in state s }\}$  and 
$M_{v_s} = \max \{v_j, \text{ for county j in state s }\}$.

We calculate the normalize value for each vulnerability for each county by substracting the min vulnerability in the state of the county and dividing that by the differences between max and min value of the vulnerability values in the state. Mathematical formulation for the normalization is as follows.

$N_{v_j} = \frac{v_j - m_{v_s}}{M_{v_s} - m_{v_s}}$ 

for each $v \in V$, where V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}, j is a county in each state $s \in S$.


In [715]:
#Normalize function to normalize the vulnerability values to be able to compare them

def normalize(dict_1):
    
    result = {}
    min_data = {s: min(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    max_data = {s: max(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    
    for (j,s) in cartesian_pro_county_state:
        
        if (max_data[s] - min_data[s]) != 0 :
    
            result[j] = (dict_1[j] - min_data[s])/(max_data[s] - min_data[s])
        
        else:
            result[j] = 1
    
    return result 

# Percentile Rank

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html"> The function scipy.stats.percentileofscore (a, score, kind='rank')   </a>
computes the percentile rank of a score relative to a list of scores. 
"rank": Average percentage ranking of score. In case of multiple matches, average the percentage rankings of all matching scores.

In [716]:
from scipy import stats

# Calculate percentile ranks

def percentile_ranks(data):
    x = {s: [] for s in State}

    for (j,s) in cartesian_pro_county_state:
         
        x[s].append(data[j])
    
    
    
    percentile_ranks = {i: stats.percentileofscore(x[s], data[i], 'rank') for (i,s) in cartesian_pro_county_state}

    return percentile_ranks

In [717]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [718]:
#print (Medicaid_demand)

In [719]:
#Write a function to order the dicts
def order_k(dict_1):
    dict_2 = {}
    for m in location:
        if m in dict_1.keys():
            dict_2[m] = dict_1[m]
        else:
            dict_2[m] = 0
    
    return dict_2
            

In [720]:


Medicaid_demand = order_k(Medicaid_demand)#{m: Medicaid_demand[m] for m in location}
COVID_14days = order_k(COVID_14days)#{m: COVID_14days[m] for m in location}
SVI_county = order_k(SVI_county)#{m: SVI_county[m] for m in location}
YPLL = order_k(YPLL)#{m: YPLL[m] for m in location}
Unemployment = order_k(Unemployment)#{m: Unemployment[m] for m in location}
Unemployment_capita = order_k(Unemployment_capita)#{m: Unemployment_capita[m] for m in location}

In [721]:
# Write file allocation with each strategies for each county 

Strategies = ["Medicaid_demand", "Medicaid_capita", "Covid", "SVI"
              , "YPLL","Unemployment", "Unemployment_capita", "Covid_capita",  "Covid_death_capita" ]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Percentile_ranks_' + s)

    
        

writefile = 'Output/County_level_proportional_allocation_for_all_policies.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_medicaid.values(),         Medicaid_demand.values(),      percentile_ranks(Medicaid_demand).values()
                   , Proportional_to_medicaid_cap.values(),     Medicaid_capita.values(),      percentile_ranks(Medicaid_capita).values()
                   , Proportional_to_covid.values(),            COVID_14days.values(),         percentile_ranks(COVID_14days).values()
                   , Proportional_to_SVI.values(),              SVI_county.values(),           percentile_ranks(SVI_county).values()
                   , Proportional_to_YPLL.values(),             YPLL.values(),                 percentile_ranks(YPLL).values()
                   , Proportional_to_unemployment.values(),     Unemployment.values(),         percentile_ranks(Unemployment).values()
                   , Proportional_to_unemployment_cap.values(), Unemployment_capita.values(),  percentile_ranks(Unemployment_capita).values()
                   , Proportional_to_covid_capita.values(),     Covid_capita.values(),         percentile_ranks(Covid_capita).values()
                   , Proportional_to_covid_death_cap.values(),  Covid_death_capita.values(),   percentile_ranks(Covid_death_capita).values() ):                    
       
        writer.writerow(row)

In [722]:
writefile = 'Output/State_level_allocation.csv'

cl = ['State', 'CHW_allocation']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(cl)
    for row in zip( State, Medicaid_demand_state.values()):
        writer.writerow(row)